In [1]:
%load_ext autoreload
%autoreload 2

import nltk
import spacy
import skweak

from textblob import TextBlob
from textblob.taggers import PatternTagger

from scripts.skweak_ner_eval import evaluate
from scripts.utils import penntreebank2universal, load_data_split, tag_all, compute_recall_num_conflicts


In [2]:
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')
nltk.download('stopwords')
!python -m spacy download en_core_web_sm
!python -m spacy download en_core_web_md
!python -m textblob.download_corpora

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /mnt/data/stephana93dm/storage/cache/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     /mnt/data/stephana93dm/storage/cache/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /mnt/data/stephana93dm/storage/cache/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 19.8 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 10.5 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
[nltk_data] Downloading package brown to
[nltk_data]     /mnt/data/stephana93dm/storage/cache/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /mnt/data/stephana93dm/storage/cache/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /mnt/data/stephana93dm/storage/cache/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /mnt/data/stephana93dm/storage/cache/nltk_data...
[nltk_data

## Using Libraries as Labelling functions


In this part, we use popular NLP libraries to create labeling functions. They include Spacy, NLTK, Textblob.
We use the Majority Voter and HMM as aggregation functions
Optionally, you can train your own model on the data.

Learning goals:
- Understand how to use external libraries as labeling functions
- Understand the Spacy object and how to use it for annotation
- Understand the impact of Majority Voter and HMM as aggregation functions, and get a feeling for their performance

First, read and understand the two functions below.

In [3]:

# Sometimes data formats (here POS tags) differ. We load the data and convert it to the format we need. 
# Surely, there is some loss of information
def nltk_tagger(doc):
    for token in doc:
        if not token.is_punct:
            # Tag token with nltk
            nltk_pos = nltk.pos_tag([token.text])[0][1]
            # Map nltk pos tags to ours
            if nltk_pos == "DT":
                yield token.i, token.i + 1, "DET"
            elif nltk_pos == "CD":
                yield token.i, token.i + 1, "NUM"
            elif nltk_pos == "NNP" or nltk_pos == "NNPS":
                yield token.i, token.i + 1, "PROPN"
            elif nltk_pos == "JJ" or nltk_pos == "JJR" or nltk_pos == "JJS":
                yield token.i, token.i + 1, "ADJ"
            elif nltk_pos == "NN" or nltk_pos == "NNS":
                yield token.i, token.i + 1, "NOUN"
            elif nltk_pos == "VB" or nltk_pos == "VBD" or nltk_pos == "VBG" or nltk_pos == "VBN" or nltk_pos == "VBP" or nltk_pos == "VBZ":
                yield token.i, token.i + 1, "VERB"


# We cn also use the Textblob library to get POS tags
# Under the hood, it uses the Pattern library. Once again, a transformation of the tag-labels is needed
def textblob_tagger(doc):
    for token in doc:
        if not token.is_punct:
            textblob_pos = TextBlob(token.text, pos_tagger=PatternTagger()).tags
            if len(textblob_pos) > 0:
                yield token.i, token.i + 1, penntreebank2universal(textblob_pos[0][1])


## Write the Spacy Labeling Functions

Use the two english Spacy models "en_core_web_sm", "en_core_web_md" to create labeling functions.
The challenge is that they use different tokens, i.e. the atomic units of a sentence. Our simple tokenization just splits the words by whitespace.
Your task it to design an algorithm that maps the tokens of the simple tokenization to the tokens of the Spacy tokenization, and use the token available there to create labeling functions.

Hints:
1) Access token i by `token=doc[i]` or obtain its poition by `i=token.i`
2) Access the Spacy POS token by `pos=token.pos_`

In [4]:
eng_nlp_sm = spacy.load("en_core_web_sm")
eng_nlp_md = spacy.load("en_core_web_md")

#########################################################
def eng_spacy_tagger_sm(doc):
    other_doc = eng_nlp_sm(doc.text)
    i = 0
    for token in doc:
        labelled = False
        for other_token in other_doc:
            if other_doc[other_token.i:].text not in doc[token.i:].text:
                continue
            if token.text in other_token.text and not labelled:
                labelled = True
                yield token.i, token.i + 1, other_token.pos_.split("-")[-1]


def eng_spacy_tagger_md(doc):
    other_doc = eng_nlp_md(doc.text)

    for token in doc:
        labelled = False
        for other_token in other_doc:
            if other_doc[other_token.i:].text not in doc[token.i:].text:
                continue
            if token.text in other_token.text and not labelled:
                labelled = True
                yield token.i, token.i + 1, other_token.pos_.split("-")[-1]

#########################################################

In [5]:

nltk_lf = skweak.heuristics.FunctionAnnotator("nltk", nltk_tagger)
textblob_lf = skweak.heuristics.FunctionAnnotator("textblob", textblob_tagger)
eng_spacy_sm_lf = skweak.heuristics.FunctionAnnotator("eng_spacy_sm", eng_spacy_tagger_sm)
eng_spacy_md_lf = skweak.heuristics.FunctionAnnotator("eng_spacy_md", eng_spacy_tagger_md)

### Load Data and apply Labeling functions

Before and after applying the labeling functions, and the aggregation functions, we compute the recall and number of conflicts.

In [6]:

# load training and test data
lfs = [nltk_lf, eng_spacy_sm_lf, textblob_lf, eng_spacy_md_lf]
all_labels = ["DET", "NUM", "PROPN", "NOUN", "ADJ"]

# small amount of data for the sake of time
train_docs = load_data_split("train", all_labels, 3000)
test_docs = load_data_split("test", all_labels, 1000)

# tag the training documents
train_docs = tag_all(train_docs, lfs)



In [7]:
recall, num_conflicts = compute_recall_num_conflicts(train_docs)
print("Train", recall, num_conflicts)

# Add Majority Vote
mv = skweak.aggregation.MajorityVoter("mv", all_labels)
train_docs = tag_all(train_docs, [mv])
recall, num_conflicts = compute_recall_num_conflicts(train_docs)
print("Train with MV", recall, num_conflicts)

Train 1.0 0.23076923076923078
Train with MV 1.0 0.23076923076923078


In [8]:
# train the HMM
hmm = skweak.aggregation.HMM("hmm", all_labels)
hmm=hmm.fit(train_docs)

# tag the test documents
# it's important to set Majority vote before HMM, otherwise Majority Vote takes the HMM predictions into account
train_docs = tag_all(train_docs, lfs + [mv, hmm])

Starting iteration 1
Number of processed documents: 1000
Number of processed documents: 2000
Finished E-step with 3000 documents
Starting iteration 2


         1     -105487.6510             +nan


Number of processed documents: 1000
Number of processed documents: 2000
Finished E-step with 3000 documents
Starting iteration 3


         2     -100521.0518       +4966.5992


Number of processed documents: 1000
Number of processed documents: 2000
Finished E-step with 3000 documents
Starting iteration 4


         3      -99870.4155        +650.6362


Number of processed documents: 1000
Number of processed documents: 2000
Finished E-step with 3000 documents
Starting iteration 5


         4      -99584.3244        +286.0912


Number of processed documents: 1000
Number of processed documents: 2000
Finished E-step with 3000 documents


         5      -99350.4815        +233.8429


In [9]:
recall, num_conflicts = compute_recall_num_conflicts(train_docs)
print("Train", recall, num_conflicts)

Train 1.0 0.23076923076923078


## Evaluation

Look at the Precision, Recall and F1-Score of the different aggregation functions. What do you observe?

In [10]:
# tag the test documents
# it's important to set Majority vote before HMM, otherwise Majority Vote takes the HMM predictions into account
test_docs = tag_all(test_docs, lfs + [mv, hmm])

recall, num_conflicts = compute_recall_num_conflicts(train_docs)
print("Train", recall, num_conflicts)

Train 1.0 0.23076923076923078


In [11]:
df = evaluate(test_docs, all_labels, [ "mv", "hmm"])

In [12]:
df

tok_precision  tok_recall  tok_f1
label    proportion model                                   
ADJ      12.8 %     hmm            0.912       0.866   0.888
                    mv             0.893       0.852   0.872
DET      17.3 %     hmm            0.950       0.996   0.972
                    mv             0.900       0.999   0.946
NOUN     37.4 %     hmm            0.936       0.859   0.896
                    mv             0.671       0.879   0.762
NUM      7.3 %      hmm            0.964       0.927   0.946
                    mv             0.969       0.848   0.904
PROPN    25.2 %     hmm            0.806       0.950   0.872
                    mv             0.808       0.947   0.872
macro               hmm            0.914       0.920   0.916
                    mv             0.848       0.905   0.876
micro               hmm            0.899       0.911   0.904
                    mv             0.783       0.911   0.842
weighted            hmm            0.905       0.911   0.908
                    mv             0.795       0.911   0.850